# ANTTquery
Recebe valores e retorna um XLSX conforme o filtro selecionado.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").appName('query_antt').getOrCreate()

antt = spark.read.option("header", "True").parquet("./interestadual/ref/parquet/")
antt.createOrReplaceTempView("antt")
# antt.printSchema()
# spark.sql("SHOW TABLES").show(10, False)

In [ ]:
def select_antt():
    # ---------------------------------------------------------------------------------------------------------
    print("*" * 100)
    print("Selecione o tipo de particionamento:\n1 | por ano / empresa / cidade\n2 | por ano / por cidade\n3 | por ano / por empresa\n4 | por cidade full (todos os anos e empresas)")
    print("*" * 100)
    select_start = (input("Digite a opção de sua seleção: "))
    # ---------------------------------------------------------------------------------------------------------
    if int(select_start) == 1: # 1 | por ano / empresa / cidade
        print(f"Sua seleção foi:\nAno(s): {select_anos}\nEmpresa(s): {select_empresas}\nCidades(s): {select_cidades}")
        print("*" * 100)
        print("Realizando a consulta.")
        for cidade in select_cidades:
            for ano in select_anos:
                for empresa in select_empresas:
                    origem = cidade
                    destino = origem
                    ano = ano
                    empresa = empresa

                    df_temp = spark.sql(f"""

                    SELECT '{origem}' AS cidade_base, ano, mes, empresa, prefixo, ori_des_localidade_nome, pax_total
                    FROM antt
                    WHERE pax_total > 0
                    AND empresa LIKE "%{empresa}%"
                    AND ano = {ano}
                    AND (ori_des_localidade_nome LIKE '%{origem}%' OR ori_des_localidade_nome LIKE '%{destino}%')
                    ORDER BY ano ASC, empresa ASC, prefixo ASC, pax_total DESC

                    """)
                    print("Gravando a consulta em formato xlsx.")
                    if origem == destino:
                        df_temp.toPandas().to_excel(f"./interestadual_filtro/{origem}_{ano}_{empresa}.xlsx", index = False)
                    else:
                        df_temp.toPandas().to_excel(f"./interestadual_filtro/{origem}_{destino}_{ano}_{empresa}.xlsx", index = False)
    # ---------------------------------------------------------------------------------------------------------
    elif int(select_start) == 2: # 2 | por cidade / por ano
        print(f"Sua seleção foi:\nAno(s): {select_anos}\nCidades(s): {select_cidades}")
        print("*" * 100)
        print("Realizando a consulta.")
        for cidade in select_cidades:
            for ano in select_anos:
                origem = cidade
                destino = origem
                ano = ano

                df_temp = spark.sql(f"""

                SELECT '{origem}' AS cidade_base, ano, mes, empresa, prefixo, ori_des_localidade_nome, pax_total
                FROM antt
                WHERE pax_total > 0
                AND ano = {ano}
                AND (ori_des_localidade_nome LIKE '%{origem}%' OR ori_des_localidade_nome LIKE '%{destino}%')
                ORDER BY ano ASC, empresa ASC, prefixo ASC, pax_total DESC

                """)
                print("Gravando a consulta em formato xlsx.")
                if origem == destino:
                    df_temp.toPandas().to_excel(f"./interestadual_filtro/{origem}_{ano}.xlsx", index = False)
                else:
                    df_temp.toPandas().to_excel(f"./interestadual_filtro/{origem}_{destino}_{ano}.xlsx", index = False)
        print("Gravação realizada.")
    # ---------------------------------------------------------------------------------------------------------
    elif int(select_start) == 3: # 3 | por ano / por empresa
        print(f"Sua seleção foi:\nAno(s): {select_anos}\nEmpresa(s): {select_empresas}")
        print("*" * 100)
        for empresa in select_empresas:
            for ano in select_anos:
                ano = ano
                empresa = empresa

                df_3 = spark.sql(f"""

                SELECT ano, mes, empresa, prefixo, ori_des_localidade_nome, pax_total
                FROM antt
                WHERE pax_total > 0
                AND empresa LIKE '%{empresa}%'
                AND ano = {ano}
                ORDER BY ano ASC, empresa ASC, prefixo ASC, pax_total DESC

                """)
                print("Gravando a consulta em formato xlsx.")

                df_3.toPandas().to_excel(f"./interestadual_filtro/{empresa}_{ano}.xlsx", index = False)
    # ---------------------------------------------------------------------------------------------------------
    elif int(select_start) == 4: # 3 | por cidade full
        print(f"Sua seleção foi:\nCidade(s): {select_cidades}")
        print("*" * 100)
        for cidade in select_cidades:
            origem = cidade
            destino = origem
            df_4 = spark.sql(f"""

            SELECT '{origem}' AS cidade_base, ano, mes, empresa, prefixo, ori_des_localidade_nome, pax_total
            FROM antt
            WHERE pax_total > 0
            AND (ori_des_localidade_nome LIKE '%{origem}%' OR ori_des_localidade_nome LIKE '%{destino}%')
            ORDER BY ano ASC, empresa ASC, prefixo ASC, pax_total DESC

            """)

            if origem == destino:
                df_4.toPandas().to_excel(f"./interestadual_filtro/{origem}_todos.xlsx", index = False)
            else:
                df_4.toPandas().to_excel(f"./interestadual_filtro/{origem}_{destino}_todos.xlsx", index = False)

        print("Gravação realizada.")
    # ---------------------------------------------------------------------------------------------------------
    else:
        print("Selecione uma opção válida de consulta.")
    
    # if empresa == "":
    #     df_temp.withColumn("cidade_base", lit(origem)).toPandas().to_excel(f"{ano}_{origem}_{destino}.xlsx", index = False)
    # else:
    #     df_temp.withColumn("cidade_base", lit(origem)).toPandas().to_excel(f"{ano}_{empresa}_{origem}_{destino}.xlsx", index = False)    
    # ---------------------------------------------------------------------------------------------------------

In [ ]:
# select_anos = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
select_anos = [2019]
select_cidades = ["JOINVILLE-SC"]
select_empresas = ["GARCIA"]

select_antt()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").appName('').getOrCreate()

In [ ]:
df = spark.read.parquet("./intermunicipal_sc/ref_join/parquet/singlefile/")

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
print(f"{df.count()} linhas | {len(df.columns)} colunas")

In [ ]:
df.printSchema()

In [ ]:
sql = spark.sql("SELECT ano, empresa, modalidade, linha, sig_secao_ori_des AS secao, km, passageiros_total AS pax_total FROM df").show(20, False)

In [ ]:
import pandas as pd
dfp = pd.read_parquet("./intermunicipal_sc/ref_join/parquet/singlefile")
dfp.head()

In [ ]:
# Script para gerar file "todos"
empresa = ""
origem = ""
destino = origem
ano = "todos"

df_temp = spark.sql(f"""

SELECT
    '{origem}' AS cidade_base,
    ano,
    empresa,
    prefixo,
    ori_des_localidade_nome,
    CAST(pax_total AS integer) AS passageiros
FROM
    antt
WHERE
    pax_total > 0
-- AND
    -- ano = {ano}
AND
    (ori_des_localidade_nome LIKE '%{origem}%' OR ori_des_localidade_nome LIKE '%{destino}%')
ORDER BY
    ano ASC,
    empresa ASC,
    prefixo ASC,
    passageiros DESC

""")

df_temp.write.option("encoding", "utf-8").parquet("./teste_pqfull/")

# if empresa == "":
#     if origem == destino:
#         df_temp.toPandas().to_excel(f"./interestadual_filtro/{origem}_{ano}.xlsx", index = False)
#     else:
#         df_temp.toPandas().to_excel(f"./interestadual_filtro/{origem}_{destino}_{ano}.xlsx", index = False)

# else:
#     if origem == destino:
#         df_temp.toPandas().to_excel(f"./interestadual_filtro/{empresa}_{origem}_{ano}.xlsx", index = False)
#     else:
#         df_temp.toPandas().to_excel(f"./interestadual_filtro/{empresa}_{origem}_{destino}_{ano}.xlsx", index = False)

In [ ]:
# schema = StructType([
#     StructField("ano", StringType(), True),
#     StructField("codigo", StringType(), True),
#     StructField("des_localidade_id", StringType(), True),
#     StructField("des_localidade_nome", StringType(), True),
#     StructField("des_localidade_uf", StringType(), True),
#     StructField("des_municipio_id", StringType(), True),
#     StructField("des_municipio_nome", StringType(), True),
#     StructField("empresa", StringType(), True),
#     StructField("empresa_cnpj", StringType(), True),
#     StructField("empresa_situacao", StringType(), True),
#     StructField("empresa_tipo", StringType(), True),
#     StructField("ida_idoso_desconto", StringType(), True),
#     StructField("ida_idoso_gratis", StringType(), True),
#     StructField("ida_jovem_desconto", StringType(), True),
#     StructField("ida_jovem_gratis", StringType(), True),
#     StructField("ida_pagantes", StringType(), True),
#     StructField("ida_passelivre", StringType(), True),
#     StructField("km", StringType(), True),
#     StructField("km_total", StringType(), True),
#     StructField("linha", StringType(), True),
#     StructField("linha_id", StringType(), True),
#     StructField("lugares_idas", StringType(), True),
#     StructField("lugares_voltas", StringType(), True),
#     StructField("mes", StringType(), True),
#     StructField("ori_des_localidade_nome", StringType(), True),
#     StructField("ori_localidade_id", StringType(), True),
#     StructField("ori_localidade_nome", StringType(), True),
#     StructField("ori_localidade_uf", StringType(), True),
#     StructField("ori_municipio_id", StringType(), True),
#     StructField("ori_municipio_nome", StringType(), True),
#     StructField("pax_gratis_descontos", StringType(), True),
#     StructField("pax_idoso_desconto", StringType(), True),
#     StructField("pax_idoso_gratis", StringType(), True),
#     StructField("pax_jovem_desconto", StringType(), True),
#     StructField("pax_jovem_gratis", StringType(), True),
#     StructField("pax_pagantes", StringType(), True),
#     StructField("pax_passelivre", StringType(), True),
#     StructField("pax_total", StringType(), True),
#     StructField("prefixo", StringType(), True),
#     StructField("secao_id", StringType(), True),
#     StructField("sequencial", StringType(), True),
#     StructField("servico_ambito", StringType(), True),
#     StructField("servico_tipo", StringType(), True),
#     StructField("sisdap_fim", StringType(), True),
#     StructField("sisdap_inicio", StringType(), True),
#     StructField("viagem_idas", StringType(), True),
#     StructField("viagem_voltas", StringType(), True),
#     StructField("volta_idoso_desconto", StringType(), True),
#     StructField("volta_idoso_gratis", StringType(), True),
#     StructField("volta_jovem_desconto", StringType(), True),
#     StructField("volta_jovem_gratis", StringType(), True),
#     StructField("volta_pagantes", StringType(), True),
#     StructField("volta_passelivre", StringType(), True)])